In [1]:
import sys
import os

root_dir = os.path.dirname(os.getcwd())
sys.path.append(root_dir)

import pandas as pd
from implicit.nearest_neighbours import CosineRecommender
from models.user_knn_with_cold_start import UserKnnWithColdStart
from models.userknn import UserKnn
from rectools.metrics import calc_metrics, Precision, Recall, Serendipity, MeanInvUserFreq, MAP, NDCG

from service.config import INTERACTIONS_DATA
import pickle

In [2]:
# Загружаем данные
interactions = pd.read_csv(INTERACTIONS_DATA)
interactions.rename(
        columns={
                'last_watch_dt': 'datetime',
                'total_dur': 'weight'
        }, 
        inplace=True)
interactions['datetime'] = pd.to_datetime(interactions['datetime'])
interactions.head()

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [3]:
# Разобьем для трейна и теста
max_date = interactions['datetime'].max()
train = interactions[(interactions['datetime'] < max_date - pd.Timedelta(days=7))]
test = interactions[(interactions['datetime'] >= max_date - pd.Timedelta(days=7))]

# Отделим на тесте теплых и холодных пользователей
warm_users = train['user_id'].unique()
warm_test = test[test['user_id'].isin(warm_users)]
cold_test = test[~test['user_id'].isin(warm_users)]

print(f"Количество теплых пользователей: {warm_test['user_id'].nunique()}")
print(f"Количество холодных пользователей: {cold_test['user_id'].nunique()}")

Количество теплых пользователей: 120519
Количество холодных пользователей: 65388


### Загрузка моделей

In [4]:
# Объявляем глобальные переменные
N_USERS = 50
N_RECS = 10

In [6]:
# Загружаем или обучаем старый формат модели
if os.path.exists(os.path.join(root_dir, 'service/saved_models/user_knn_old.pkl')):
    with open(os.path.join(root_dir, 'service/saved_models/user_knn_old.pkl'), 'rb') as f:
        userknn = pickle.load(f)
else:
    model = CosineRecommender(N_USERS)
    userknn = UserKnn(model, N_USERS)
    userknn.fit(train)
    with open(os.path.join(root_dir, 'service/saved_models/user_knn_old.pkl'), 'wb') as f:
        pickle.dump(userknn, f)

In [7]:
# Загружаем или обучаем новый формат модели
if os.path.exists(os.path.join(root_dir, 'service/saved_models/user_knn_with_cold_start.pkl')):
    with open(os.path.join(root_dir, 'service/saved_models/user_knn_with_cold_start.pkl'), 'rb') as f:
        userknn_with_cold_start = pickle.load(f)
else:
    userknn_with_cold_start = UserKnnWithColdStart(N_USERS, N_RECS)
    userknn_with_cold_start.fit(train)
    with open(os.path.join(root_dir, 'service/saved_models/user_knn_with_cold_start.pkl'), 'wb') as f:
        pickle.dump(userknn_with_cold_start, f)

Отличие нового формата от старого:
- количество рекомендаций строго N
- для холодных пользователей реализованы предсказания топа популярных
- метод recommend реализован не для батчевого предикта, а для поюзерного

### Оценка качества модели

In [8]:
# Делаем предсказания по теплым пользователям и холодным с помощью старой модели
warm_recommends = userknn.predict(warm_test)[['user_id', 'item_id', 'rank']]

# Проверка, что по холодным пользователям не работает
cold_recommends = userknn.predict(cold_test)

KeyError: 505244

In [19]:
# Делаем предсказания по Топ Популярным
top_popular_recs = pd.DataFrame(test['user_id'].unique(), columns=['user_id'])
top_popular_recs['item_id'] = [list(top_popular.recommend()[:N_RECS])] * top_popular_recs.shape[0]
top_popular_recs = top_popular_recs.explode(column='item_id').reset_index(drop=True)
top_popular_recs['rank'] = top_popular_recs.groupby('user_id').cumcount()
top_popular_recs.head()

,user_id,item_id,rank
0,203219,10440,0
1,203219,15297,1
2,203219,9728,2
3,203219,13865,3
4,203219,4151,4


In [21]:
# Делаем предсказания по теплым пользователям и холодным с помощью новой модели
new_warm_recommends = []
for warm_user in warm_users:
    items = userknn_with_cold_start.recommend(warm_user)
    for i, item in enumerate(items):
        curr_rec = {
            'user_id': warm_user,
            'item_id': item,
            'rank': i + 1
        }
        new_warm_recommends.append(curr_rec)


new_cold_recommends = []
for cold_user in cold_test['user_id'].unique():
    items = userknn_with_cold_start.recommend(cold_user)
    for i, item in enumerate(items):
        curr_rec = {
            'user_id': warm_user,
            'item_id': item,
            'rank': i + 1
        }
        new_cold_recommends.append(curr_rec)

new_warm_recommends_df = pd.DataFrame(new_warm_recommends)
new_cold_recommends_df = pd.DataFrame(new_cold_recommends)


In [22]:
# Проверка, что всегда отдаем по 10 рекомендациям
users_wo_need_recs_cnt = (new_warm_recommends_df.groupby('user_id')['rank'].max() < 10).sum()
print(f'Количество пользователей, у которых меньше 10 рекомендаций: {users_wo_need_recs_cnt}')

Количество пользователей, у которых меньше 10 рекомендаций: 0
